# **Simulasi dan Animasi Chord DHT dengan Finger Table**

- **Penulis**: `Rozhak`
- **Versi**: `1.0`
- **Pembaruan**: `27 Oktober 2025`

## **1. Persiapan Environment dan Impor Pustaka**

Pertama kita mengimpor semua pustaka yang diperlukan untuk simulasi dan visualisasi. Pustaka utama yang digunakan adalah `matplotlib` untuk plotting dan animasi, `numpy` untuk perhitungan numerik, serta `IPython` untuk menampilkan animasi di notebook.

In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from IPython.display import HTML
import matplotlib

matplotlib.use('agg')

## **2. Inisialisasi Node dan Ring DHT**

Pada bagian ini kita menyiapkan struktur dasar Chord DHT dengan menentukan jumlah node dan posisi mereka dalam ring. Setiap node ditempatkan secara merata pada lingkaran dengan koordinat yang dihitung menggunakan fungsi trigonometri.

In [2]:
num_nodes = 8
angles = np.linspace(0, 2 * np.pi, num_nodes, endpoint=False)
x = np.cos(angles)
y = np.sin(angles)

nodes = list(range(num_nodes))

## **3. Konfigurasi Lookup dan Hash Function**

Di sini kita mendefinisikan key yang akan dicari dan node awal pencarian. Fungsi hash sederhana digunakan untuk memetakan key ke node tertentu dalam ring. Key 150 akan di-hash untuk menentukan node tujuan.

In [3]:
key_to_lookup = 150
start_node = 0

def simple_hash(key):
    """Fungsi hash sederhana untuk memetakan key ke node dalam ring.

    Args:
        key (int): Key yang akan di-hash.

    Returns:
        int: Node yang sesuai dengan key.
    """
    return key % num_nodes

key_node = simple_hash(key_to_lookup)

## **4. Konstruksi Finget Table**

Finger table adalah komponen kunci dalam Chord DHT yang mempercepat proses lookup. Setiap node menyimpan referensi ke node-node yang berjarak $2^k$ untuk $k = 0, 1, 2$. Ini memungkinkan pencarian yang efisien dengan kompleksitas $O(\log N)$.

In [4]:
finger_table = {i: [(i + 2**k) % num_nodes for k in range(3)] for i in nodes}

## **5. Simulasi Proses Lookup**

Algoritma lookup mensimulasikan bagaimana sebuah key ditemukan dalam DHT. Proses dimulai dari start node dan melompat menggunakan finger table hingga mencapai node yang bertanggung jawab untuk key tersebut.

In [5]:
lookup_path = [start_node]
current_node = start_node

while current_node != key_node:
    next_node = current_node
    for f in sorted(finger_table[current_node], reverse=True):
        if (f > current_node and f <= key_node) or (current_node > key_node and (f > current_node or f <= key_node)):
            next_node = f
            break

    if next_node == current_node:
        next_node = (current_node + 1) % num_nodes
    lookup_path.append(next_node)
    current_node = next_node

print("Lookup Path:", lookup_path)

Lookup Path: [0, 4, 6]


## **6. Setup Visualisasi dan Animasi**

Kita menyiapkan canvas untuk visualisasi dengan membuat ring DHT dan menempatkan semua node. Setiap node akan ditampilkan sebagai titik biru dengan label N0, N1, dst.

In [6]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(-1.2, 1.2)
ax.set_ylim(-1.2, 1.2)
ax.axis('off')

ring = plt.Circle((0, 0), 1, color='lightgray', fill=False, linestyle='--')
ax.add_artist(ring)

scat = ax.scatter(x, y, color='deepskyblue', s=200, zorder=3)
texts = [ax.text(x[i]*1.1, y[i]*1.1, f'N{nodes[i]}', fontsize=10,
         ha='center', va='center') for i in range(num_nodes)]

(key_dot,) = ax.plot([], [], 'ro', markersize=10, label='Key Position')
(path_line,) = ax.plot([], [], 'r--', lw=2, alpha=0.7)

step_text = ax.text(0, -1.35, '', ha='center', fontsize=12, color='black',
                    bbox=dict(facecolor='white', alpha=0.8, edgecolor='none',
                              boxstyle='round,pad=0.5'))

## **7. Fungsi Animasi Lookup**

Fungsi animasi mengontrol bagaimana visualisasi berkembang seiring waktu. Setiap frame menujukkan satu langkah dalam proses lookup, dengan path yang menghubungkan node-node yang dikujungi.

In [7]:
def init():
    """Inisialisasi animasi - reset plot.

    Menyiapkan frame awal animasi dengan mengosongkan data pada
    titik key, garis path, dan teks langkah.

    Returns:
        tuple: Objek-objek matplotlib yang diperbarui.
    """
    key_dot.set_data([], [])
    path_line.set_data([], [])
    step_text.set_text('')

    return key_dot, path_line, step_text

def animate(i):
    """Fungsi animasi per frame.

    Memperbarui plot untuk setiap langkah dalam simulasi lookup Chord DHT.

    Args:
        i (int): Indeks frame saat ini.

    Returns:
        tuple: Objek-objek matplotlib yang diperbarui.
    """
    key_dot.set_data([x[key_node]], [y[key_node]])
    key_dot.set_zorder(5)

    if i < len(lookup_path) - 1:
        path_x = [x[node] for node in lookup_path[: i + 2]]
        path_y = [y[node] for node in lookup_path[: i + 2]]

        path_line.set_data(path_x, path_y)
        step_text.set_text(f'Step {i+1}: N{lookup_path[i]} → N{lookup_path[i+1]}')
    elif i == len(lookup_path) - 1:
        path_x = [x[node] for node in lookup_path]
        path_y = [y[node] for node in lookup_path]

        path_line.set_data(path_x, path_y)
        step_text.set_text(f'Key ditemukan di N{key_node}')

    return key_dot, path_line, step_text

## **8. Generate dan Tampilkan Animasi**

Terakhir kita membuat animasi dengan mengatur interval dan parameter lainnya, kemudian menyampaikan sebagai HTML video di notebook.

In [8]:
ani = animation.FuncAnimation(fig, animate, frames=len(lookup_path),
                              init_func=init, blit=True, interval=1200, repeat=False)

plt.title('Simulasi Lookup Chord DHT (Finger Table)', fontsize=14)
plt.legend(loc='upper right')
plt.close(fig)

js_ani = HTML(ani.to_jshtml())
display(js_ani)

In [9]:
ani.save('chord_dht.gif')